In [ ]:
import pandas as pd
data = pd.read_csv('CSV TSE/1 - PRESIDENTE/gtm2023_e1.csv', skiprows=range(0,4), header=1)
data.head()

,MESA,DEPARTAMENTO,ID_DEPARTAMENTO,MUNICIPIO,ID_MUNICIPIO,CENTRO_DE_VOTACIÓN,PADRÓN,CUARENTA Y UNO,QUINCE,TREINTA Y DOS,...,IMPUGNACIONES,PRES-VOTAR,VALIDOS_CALCULADO,EMITIDOS_CALCULADO,TOTAL_CALCULADO,OBSERVACIONES,CONTABILIZADA,CÓDIGO_INTEGRIDAD,FECHA_HORA_PUBLICACIÓN,DIGITALIZACIÓN
0,1,Distrito Central,0,Guatemala,1,1,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,2,Distrito Central,0,Guatemala,1,1,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,3,Distrito Central,0,Guatemala,1,1,277,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,4,Distrito Central,0,Guatemala,1,1,277,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,5,Distrito Central,0,Guatemala,1,2,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.drop(['ID_DEPARTAMENTO','ID_MUNICIPIO','CENTRO_DE_VOTACIÓN','PADRÓN','CÓDIGO_INTEGRIDAD','FECHA_HORA_PUBLICACIÓN','DIGITALIZACIÓN'], axis=1)
data.head()

,MESA,DEPARTAMENTO,ID_DEPARTAMENTO,MUNICIPIO,ID_MUNICIPIO,CENTRO_DE_VOTACIÓN,PADRÓN,CUARENTA Y UNO,QUINCE,TREINTA Y DOS,...,IMPUGNACIONES,PRES-VOTAR,VALIDOS_CALCULADO,EMITIDOS_CALCULADO,TOTAL_CALCULADO,OBSERVACIONES,CONTABILIZADA,CÓDIGO_INTEGRIDAD,FECHA_HORA_PUBLICACIÓN,DIGITALIZACIÓN
0,1,Distrito Central,0,Guatemala,1,1,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,2,Distrito Central,0,Guatemala,1,1,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,3,Distrito Central,0,Guatemala,1,1,277,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,4,Distrito Central,0,Guatemala,1,1,277,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,5,Distrito Central,0,Guatemala,1,2,400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
